This notebook is developed using the `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` kernel on an `ml.t3.medium` instance.

In [ ]:
!pip install -q sagemaker-experiments

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter09'

In [ ]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.datasets import imdb

In [ ]:
max_features = 20000
maxlen = 400

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
data_dir = os.path.join(os.getcwd(), 'imdb_data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(data_dir, 'train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(data_dir, 'test')
os.makedirs(test_dir, exist_ok=True)

csv_test_dir = os.path.join(data_dir, 'csv-test')
os.makedirs(csv_test_dir, exist_ok=True)

np.save(os.path.join(train_dir, 'x_train.npy'), x_train)
np.save(os.path.join(train_dir, 'y_train.npy'), y_train)
np.save(os.path.join(test_dir, 'x_test.npy'), x_test)
np.save(os.path.join(test_dir, 'y_test.npy'), y_test)
np.savetxt(os.path.join(csv_test_dir, 'csv-test.csv'), 
           np.array(x_test[:100], dtype=np.int32), fmt='%d', delimiter=",")

In [ ]:
traindata_s3_prefix = f'{prefix}/imdb_data/train'
testdata_s3_prefix = f'{prefix}/imdb_data/test'

train_s3 = sess.upload_data(path='./imdb_data/train/', key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./imdb_data/test/', key_prefix=testdata_s3_prefix)

In [ ]:
!mkdir code

In [ ]:
%%writefile code/smdp_tensorflow_sentiment.py
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
import argparse
import codecs
import json
import numpy as np
import os
import tensorflow as tf

import smdistributed.dataparallel.tensorflow as sdp

max_features = 20000
maxlen = 400
embedding_dims = 300
filters = 256
kernel_size = 3
hidden_dims = 256

def parse_args():
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument('--epochs', type=int, default=1)
    parser.add_argument('--batch_size', type=int, default=64)
    parser.add_argument('--learning_rate', type=float, default=0.01)
    parser.add_argument('--drop_out_rate', type=float, default=0.2)

    # data directories
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))

    # model directory /opt/ml/model default set by SageMaker
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))

    return parser.parse_known_args()


def get_train_data(train_dir, batch_size):
    x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
    print(f'x train {x_train.shape} y train {y_train.shape}')
    
    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset


def get_test_data(test_dir):
    x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
    y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
    print(f'x test {x_test.shape} y test {y_test.shape}')

    return x_test, y_test


def get_model(args):
    embedding_layer = tf.keras.layers.Embedding(max_features,
                                                embedding_dims,
                                                input_length=maxlen)

    sequence_input = tf.keras.Input(shape=(maxlen,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    x = tf.keras.layers.Dropout(args.drop_out_rate)(embedded_sequences)
    x = tf.keras.layers.Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1)(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
    x = tf.keras.layers.Dense(hidden_dims, activation='relu')(x)
    x = tf.keras.layers.Dropout(args.drop_out_rate)(x)
    preds = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(sequence_input, preds)

    return model


def train(train_dataset, args):
    model = get_model(args)
    
    loss = tf.losses.BinaryCrossentropy(name = 'binary_crossentropy')
    acc = tf.metrics.BinaryAccuracy(name = 'accuracy')
    optimizer = tf.optimizers.Adam(learning_rate = args.learning_rate)
    
    @tf.function
    def training_step(x_train, y_train, first_batch):
        with tf.GradientTape() as tape:
            probs = model(x_train, training=True)
            loss_value = loss(y_train, probs)
            acc_value = acc(y_train, probs)

        # SMDataParallel: Wrap tf.GradientTape with SMDataParallel's DistributedGradientTape
        tape = sdp.DistributedGradientTape(tape, sparse_as_dense = True)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if first_batch:
            print('first batch')
            # SMDataParallel: Broadcast model and optimizer variables
            sdp.broadcast_variables(model.variables, root_rank=0)
            sdp.broadcast_variables(optimizer.variables(), root_rank=0)

        # SMDataParallel: all_reduce call
        loss_value = sdp.oob_allreduce(loss_value)  # Average the loss across workers
        acc_value = sdp.oob_allreduce(acc_value)
 
        return loss_value, acc_value
    
    for epoch in range(args.epochs):
        for batch, (x_train, y_train) in enumerate(train_dataset.take(len(train_dataset)//sdp.size())):
            is_first_batch = (epoch == 0) and (batch == 0)
            loss_value, acc_value = training_step(x_train, y_train, is_first_batch)

            if batch % 10 == 0 and sdp.rank() == 0:
                print('Epoch #%d, Step #%d\tLoss: %.6f, Acc: %.6f (batch_size=%d)' % (epoch, batch, loss_value, acc_value, len(y_train)))

    # SMDataParallel: Save checkpoints only from master node.
    if sdp.rank() == 0:
        model.save(os.path.join(args.model_dir, '1'))
    

if __name__ == "__main__":

    args, _ = parse_args()

    # initialize sagemaker data parallel (dist)
    sdp.init()

    # ping each GPU to a single smdistributed.dataparallel process with local_rank
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    if gpus:
        tf.config.experimental.set_visible_devices(gpus[sdp.local_rank()], 'GPU')
        
    # scale the learning rate by number of workers
    print('sdp.size() = %s' % sdp.size())
    args.learning_rate = args.learning_rate * sdp.size()
    
    train_dataset = get_train_data(args.train, args.batch_size)

    train(train_dataset, args)


In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError

experiment_name = 'imdb-sentiment-analysis'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training a sentiment classification model using imdb dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')

In [ ]:
from sagemaker.tensorflow import TensorFlow
from time import gmtime, strftime
import time

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'imdb-smdp-tf-{exp_datetime}'

s3_output_location = f's3://{bucket}/{prefix}/{jobname}'
code_dir = f's3://{bucket}/{prefix}/{jobname}'

# SMDP supports ml.p4d.24xlarge, ml.p3dn.24xlarge, and ml.p3.16xlarge
train_instance_type = 'ml.p3.16xlarge'
hyperparameters = {'epochs': 30, 'batch_size': 512, 
                   'learning_rate': 0.001, 'drop_out_rate': 0.2}

distribution = {'smdistributed': {'dataparallel': {'enabled': True}}}

estimator = TensorFlow(source_dir='code',
                       entry_point='smdp_tensorflow_sentiment.py',
                       output_path=s3_output_location,
                       code_location=code_dir,
                       instance_type=train_instance_type,
                       instance_count=1,
                       enable_sagemaker_metrics=True,
                       hyperparameters=hyperparameters,
                       sagemaker_session=sess,
                       role=role,
                       framework_version='2.4',
                       py_version='py37', 
                       distribution=distribution)

data_channels = {'train':train_s3, 'test': test_s3}
print(data_channels)

In [ ]:
# Creating a new trial for the experiment
exp_trial = Trial.create(experiment_name=experiment_name, 
                         trial_name=jobname)

experiment_config={'ExperimentName': experiment_name,
                   'TrialName': exp_trial.trial_name,
                   'TrialComponentDisplayName': 'Training'}

estimator.fit(inputs=data_channels,
              job_name=jobname,
              experiment_config=experiment_config,
              wait=True)